In [1]:
# Based on the class defined above, we create training and validation datasets.
from transformers import DetrFeatureExtractor
from lib.DETR import CocoDetection
feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50")

feature_extractor.max_size = 256
feature_extractor.size = 128

DATA_BASE = 'data/custom/'
train_dataset = CocoDetection(img_folder=f'{DATA_BASE}/train', feature_extractor=feature_extractor)
val_dataset = CocoDetection(img_folder=f'{DATA_BASE}/val', feature_extractor=feature_extractor, train=False)

from torch.utils.data import DataLoader

def collate_fn(batch):
  pixel_values = [item[0] for item in batch]
  encoding = feature_extractor.pad_and_create_pixel_mask(pixel_values, return_tensors="pt")
  labels = [item[1] for item in batch]
  batch = {}
  batch['pixel_values'] = encoding['pixel_values']
  batch['pixel_mask'] = encoding['pixel_mask']
  batch['labels'] = labels
  return batch


def to_jax(batch):
    batch['pixel_values'] = jnp.array(batch['pixel_values'].numpy().transpose(0,2,3,1))
    batch['pixel_mask'] = jnp.array(batch['pixel_mask'])
    batch['labels'] = [{k: jnp.array(v) for k,v in n.items()} for n in batch['labels']]
    return batch

train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=2, shuffle=True, num_workers = 3)
val_dataloader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=2, shuffle=True, num_workers = 3)
cats = val_dataset.coco.cats
# Use this for the # classes
id2label = {k: v['name'] for k,v in cats.items()}

t_it = iter(train_dataloader)
batch = next(t_it)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
